In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import cloudpickle as pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

## Configure your GPU for large calcul
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

## After checking     Trying it first
dense_layers = [0]   # [0, 1, 2]
layer_sizes = [64]   # [32, 64, 128]
conv_layers = [3]    # [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            ## TensorBoard
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])
            
#### Save model
model.save("64x3-CNN.model")

3-conv-64-nodes-0-dense-1541013004
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 438s 25ms/step - loss: 0.6274 - acc: 0.6361 - val_loss: 0.5692 - val_acc: 0.7106
Epoch 2/10
17462/17462 [==============================] - 424s 24ms/step - loss: 0.5194 - acc: 0.7458 - val_loss: 0.5156 - val_acc: 0.7509
Epoch 3/10
17462/17462 [==============================] - 433s 25ms/step - loss: 0.4611 - acc: 0.7844 - val_loss: 0.4588 - val_acc: 0.7791
Epoch 4/10
17462/17462 [==============================] - 458s 26ms/step - loss: 0.4150 - acc: 0.8100 - val_loss: 0.4206 - val_acc: 0.8064
Epoch 5/10
17462/17462 [==============================] - 439s 25ms/step - loss: 0.3751 - acc: 0.8305 - val_loss: 0.4057 - val_acc: 0.8165
Epoch 6/10
17462/17462 [==============================] - 424s 24ms/step - loss: 0.3433 - acc: 0.8465 - val_loss: 0.3933 - val_acc: 0.8199
Epoch 7/10
17462/17462 [==============================] - 446s 26ms/step - loss: 0